In [3]:
import pandas as pd
import io

In [2]:
df = pd.read_csv(r'norm_IndovestDKG_encoded_indovest.csv')
df.head()

,subject_entity_id,relation_id,object_entity_id,temporal_id
0,0,0,10203,0
1,1,1,244,0
2,1,1,16606,0
3,1,1,16607,0
4,1,1,3573,0


In [5]:
def split_dataframe_temporal_by_ratio(df, temporal_column, train_ratio=0.8, val_ratio=0.1, test_ratio=0.1):
    if not (0 <= train_ratio <= 1 and 0 <= val_ratio <= 1 and 0 <= test_ratio <= 1):
        raise ValueError("Rasio (train_ratio, val_ratio, test_ratio) harus antara 0 dan 1.")
    if (train_ratio + val_ratio + test_ratio) > 1.001:
        print("Peringatan: Jumlah rasio melebihi 1.0. Set test akan disesuaikan untuk memastikan total 100%.")

    df_sorted = df.sort_values(by=temporal_column).reset_index(drop=True)

    total_size = len(df_sorted)
    
    train_size = int(total_size * train_ratio)
    val_size = int(total_size * val_ratio)
    
    test_size = total_size - train_size - val_size 

    train_df = df_sorted.iloc[:train_size]
    val_df = df_sorted.iloc[train_size : train_size + val_size]
    test_df = df_sorted.iloc[train_size + val_size : train_size + val_size + test_size]

    return train_df, val_df, test_df

df = pd.read_csv(r'norm_IndovestDKG_encoded_indovest.csv')

TRAIN_RATIO = 0.80
VALID_RATIO = 0.10
TEST_RATIO = 0.10 

train_df, val_df, test_df = split_dataframe_temporal_by_ratio(
    df,
    temporal_column='temporal_id',
    train_ratio=TRAIN_RATIO,
    val_ratio=VALID_RATIO,
    test_ratio=TEST_RATIO
)

print(f"Total data dalam DataFrame awal: {len(df)}")
print("---")
print(f"Training Set: {len(train_df)} ({len(train_df)/len(df)*100:.2f}%)")
print(f"Validation Set: {len(val_df)} ({len(val_df)/len(df)*100:.2f}%)")
print(f"Test Set: {len(test_df)} ({len(test_df)/len(df)*100:.2f}%)")
print("---")


Total data dalam DataFrame awal: 68207
---
Training Set: 54565 (80.00%)
Validation Set: 6820 (10.00%)
Test Set: 6822 (10.00%)
---


In [6]:
train_df.to_csv(r'splited/train.csv', index=False)
val_df.to_csv(r'splited/valid.csv', index=False)
test_df.to_csv(r'splited/test.csv', index=False)

# formating

In [7]:
import os

In [ ]:
base_relative_dir = r'experiments/result/splited'
encoded_files_relative_dir = r'.' 

output_dir_name = r'final_txt_format'
output_dir = os.path.join(base_relative_dir, output_dir_name) # Menyimpan output di subfolder dari base_relative_dir
os.makedirs(output_dir, exist_ok=True)

files_to_convert = ['train.csv', 'valid.csv', 'test.csv']
column_names_for_split_files = ['subject_entity_id', 'relation_id', 'object_entity_id', 'temporal_id']

for file_name in files_to_convert:
    input_path = os.path.join(base_relative_dir, file_name)
    output_path = os.path.join(output_dir, file_name.replace('.csv', '.txt'))
    
    df = pd.read_csv(input_path, header=None, names=column_names_for_split_files)
    df.to_csv(output_path, sep=' ', index=False, header=False)

entity_input_path = os.path.join(encoded_files_relative_dir, 'norm_IndovestDKG_encoded_entity_id.csv')
entity_output_path = os.path.join(output_dir, 'entity2id.txt')

df_entity = pd.read_csv(entity_input_path)
with open(entity_output_path, 'w', encoding='utf-8') as f:
    for index, row in df_entity.iterrows():
        f.write(f"{row['entity_name']}\t{row['entity_id']}\n")

relation_input_path = os.path.join(encoded_files_relative_dir, 'norm_IndovestDKG_relation_id.csv')
relation_output_path = os.path.join(output_dir, 'relation2id.txt')

df_relation = pd.read_csv(relation_input_path)
with open(relation_output_path, 'w', encoding='utf-8') as f:
    for index, row in df_relation.iterrows():
        f.write(f"{row['relation']}\t{row['relation_id']}\n")

FileNotFoundError: [Errno 2] No such file or directory: 'experiments\\result\\splited\\train.csv'